<a href="https://colab.research.google.com/github/Pilo1961/Deep_Learning/blob/master/Copia_de_StackGan_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# importa modulos propios
import sys
sys.path.append('/content/drive/My Drive/')

import model
#import util

Using TensorFlow backend.


In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.transform import rescale
from skimage import img_as_ubyte
import pickle
import datetime as dt

Especificamo hiperparametros

In [0]:
data_dir = '/content/drive/My Drive'
train_dir = data_dir + "/train_64"
test_dir = data_dir + "/test_64"
image_size = 64
batch_size = 64
z_dim = 100
stage1_generator_lr = 0.0002
stage1_discriminator_lr = 0.0002
stage1_lr_decay_step = 600
epochs = 1000
condition_dim = 128

#embeddings_file_path_train = train_dir + "/char-CNN-RNN-embeddings.pickle"
#embeddings_file_path_test = test_dir + "/char-CNN-RNN-embeddings.pickle"

#filenames_file_path_train = train_dir + "/filenames.pickle"
#filenames_file_path_test = test_dir + "/filenames.pickle"

#class_info_file_path_train = train_dir + "/class_info.pickle"
#class_info_file_path_test = test_dir + "/class_info.pickle"

#cub_dataset_dir = data_dir + "/CUB_200_2011"

# Load data
## Image list

In [0]:
#esto se debe de ir al modulo util
def add_fileName(df):
  '''
    Add filename column to the ID-description list
  '''
  df['filename']='a'
  for index, row in df.iterrows():
      try:
        new_name = row['ID'][:-6] + '_' + row['ID'][-1] + '.jpg'
        row['filename']=new_name
      except:
        found_n.append(row['ID'])

  return df

def train_test(df):
  x_train = df[df.index % 5 != 0]     # Excludes every 5th row starting from 0
  x_test = df[df.index % 5 == 0]      # Selects every 5th row starting from 0
  #print(x_train.shape)
  #print(x_test.shape)  
  return x_train, x_test


# Load image
def load_image(img_id,src='Flicker8k_Dataset/'):
    I = imread('/content/drive/My Drive/'+src+img_id)
    #I = margin_img(I)
    return I

# Hacemos un pickle que tiene un arreglo de numpy con toda la informacion de las imagenes
# ojo:
#file not found:  2258277193_586949ec62.j_1.jpg
#file not found:  2258277193_586949ec62.j_2.jpg
#file not found:  2258277193_586949ec62.j_3.jpg
#file not found:  2258277193_586949ec62.j_4.jpg

def img_train_pickle(x_train):
  src='test_64/'
  img_train=[]
  print("Images to load: ", len(x_train["filename"]))
  for i, img_name in enumerate(x_train["filename"]):
    try:
      img=load_image(img_name,src)
      img_train.append(img)
    except:
      print("file not found: ",img_name)
    if i%3000==0: print("Loaded images: ",i)
  
  img_train=np.array(img_train)
  
  with open("/content/drive/My Drive/img_test_data.pkl", 'wb') as f:
    pickle.dump(img_train,f) 


In [6]:
# Load data
df = pd.read_csv("/content/drive/My Drive/Flickr8k.token.txt", sep='\t', names=['ID', 'Text'], index_col=False)
df=add_fileName(df)
print(df.shape)

# Remove entries with nan values
df.dropna(inplace=True)
df.isnull().values.any()
df.head()

#Quito los registros de una imagen que falta
df.drop([6730,6731,6732,6733,6734],inplace=True)
df.reset_index()
df.shape

(40460, 3)


(40455, 3)

# Embedding

In [0]:
import pickle
with open("/content/drive/My Drive/embeddings_jp/infersent_1024_encoding.pkl", 'rb') as f:
  embedding = pickle.load(f) 

In [8]:
print(embedding.shape)
embedding[:5]

(32364, 1024)


array([[0.00719924, 0.29595286, 0.25099796, ..., 0.5558008 , 0.29030454,
        0.28820118],
       [0.5907835 , 0.18375209, 0.30119446, ..., 0.867978  , 0.        ,
        0.08150935],
       [0.42197692, 0.15705526, 0.19881831, ..., 0.8628484 , 0.07011686,
        0.08546458],
       [0.        , 0.1086157 , 0.24848017, ..., 0.5315687 , 0.25133806,
        0.24955702],
       [0.        , 0.12755851, 0.31044996, ..., 0.6411171 , 0.11928789,
        0.8201335 ]], dtype=float32)

In [62]:
df_emb=pd.DataFrame(embedding)
df_emb.head()

#solo correr esto si se necesita borrar la foto que no esta
#Quito los registros de una imagen que falta
#df_emb.drop([6730,6731,6732,6733,6734],inplace=True)
#df_emb.reset_index()
#df_emb.shape

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,1.746836,0.0,0.931276,1.447990,1.356762,0.859256,1.282008,1.385121,0.0,1.563787,1.454054,0.0,0.0,0.683455,0.0,0.750486,0.0,1.337395,0.639892,1.518962,0.053421,1.715396,1.283217,1.541835,1.218065,0.224575,0.792906,1.139164,0.052395,0.704676,1.610687,0.451876,0.839882,0.427703,0.858548,0.652197,1.294270,1.595016,0.978071,...,0.684472,0.0,0.0,0.0,0.974220,1.389602,0.356755,0.0,0.0,0.801388,0.0,1.224353,0.531555,1.355668,0.439719,1.063712,1.217662,0.0,0.899856,1.342100,0.0,0.000000,0.758532,0.0,1.292588,1.471911,0.0,1.148974,0.0,1.275467,0.0,1.097314,0.0,1.740773,0.468888,1.299500,0.926064,1.264413,0.0,0.813168
1,0.0,2.087718,0.0,1.209995,1.819482,1.624871,1.034178,1.612795,1.738789,0.0,2.029149,1.763703,0.0,0.0,0.818733,0.0,0.908369,0.0,1.632864,0.773741,1.784528,0.017366,2.041179,1.505827,1.960786,1.596756,0.242113,0.999698,1.410002,0.067680,0.876008,1.885961,0.540933,1.076209,0.493560,0.937256,0.942777,1.687483,1.907183,1.297547,...,0.841416,0.0,0.0,0.0,1.137444,1.647426,0.587025,0.0,0.0,0.838547,0.0,1.505699,0.635895,1.663432,0.493379,1.366974,1.302853,0.0,1.066198,1.698142,0.0,0.000000,0.822328,0.0,1.486578,1.935515,0.0,1.462019,0.0,1.440314,0.0,1.382626,0.0,2.259202,0.593171,1.575310,1.160255,1.512525,0.0,0.966996
2,0.0,2.051764,0.0,1.166297,1.722983,1.498636,0.951488,1.551099,1.629975,0.0,1.938318,1.621363,0.0,0.0,0.731320,0.0,0.904826,0.0,1.485968,0.702762,1.617838,0.000000,1.881845,1.420786,1.809913,1.454543,0.246779,0.994074,1.356247,0.090851,0.873286,1.856803,0.503457,1.110348,0.515202,0.857243,0.873388,1.420393,1.718299,1.259737,...,0.707185,0.0,0.0,0.0,1.130338,1.550914,0.597504,0.0,0.0,0.900907,0.0,1.403504,0.676798,1.556562,0.498200,1.291022,1.221080,0.0,0.988663,1.615499,0.0,0.000000,0.810549,0.0,1.518305,1.751477,0.0,1.418914,0.0,1.398941,0.0,1.327650,0.0,2.110633,0.564648,1.594213,1.078357,1.440756,0.0,0.951599
3,0.0,2.464272,0.0,1.357983,1.922686,1.777479,0.997722,1.896735,1.889828,0.0,2.132794,1.997364,0.0,0.0,0.847513,0.0,0.946881,0.0,1.837438,0.904739,2.165217,0.033110,2.309935,1.757875,1.928151,1.643613,0.219728,0.917767,1.531009,0.000000,1.002409,2.104956,0.417289,1.095618,0.464313,1.107965,0.953642,1.884227,2.034115,1.235775,...,0.935780,0.0,0.0,0.0,1.038223,1.810159,0.529604,0.0,0.0,1.005254,0.0,1.644976,0.613825,1.618092,0.424530,1.285653,1.538418,0.0,1.166991,1.931877,0.0,0.000000,1.038275,0.0,1.573743,2.078988,0.0,1.579780,0.0,1.516795,0.0,1.488402,0.0,2.503132,0.537330,1.592172,1.049968,1.648481,0.0,0.835587
4,0.0,2.688271,0.0,1.541515,2.034183,1.943247,1.120744,2.099424,1.916724,0.0,2.281366,2.043264,0.0,0.0,0.664024,0.0,1.031136,0.0,2.037824,1.045331,2.171372,0.000000,2.349773,1.871091,2.087316,1.662914,0.319845,1.050390,1.747847,0.000000,1.081504,2.164313,0.308654,1.375295,0.511467,1.117464,0.888759,1.980708,2.178228,1.314319,...,0.891186,0.0,0.0,0.0,1.250742,1.849244,0.617844,0.0,0.0,0.939353,0.0,1.940879,0.675314,1.699824,0.406623,1.283965,1.594314,0.0,1.195205,2.059218,0.0,0.181809,1.106166,0.0,1.886027,2.075312,0.0,1.993412,0.0,1.619841,0.0,1.649752,0.0,2.571683,0.588482,1.670402,1.154218,1.618534,0.0,0.991953


# Train test split

In [9]:
x_train, x_test= train_test(df)
print(x_train.shape)
print(x_test.shape)
x_train.head()

(32364, 3)
(8091, 3)


,ID,Text,filename
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .,1000268201_693b08cb0e_1.jpg
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .,1000268201_693b08cb0e_2.jpg
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...,1000268201_693b08cb0e_3.jpg
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...,1000268201_693b08cb0e_4.jpg
6,1001773457_577c3a7d70.jpg#1,A black dog and a tri-colored dog playing with...,1001773457_577c3a7d70_1.jpg


In [51]:
emb_train, emb_test= train_test(df_emb)
print(emb_train.shape)
print(emb_test.shape)

(32364, 4096)
(8091, 4096)


# Prepara imagenes

In [0]:
  with open("/content/drive/My Drive/img_train_data.pkl", 'rb') as f:
    img_train=pickle.load(f) 

In [11]:
len(img_train)

32364

In [12]:
# Tenemos las dimensiones correctas
len(img_train)

32364

# Creating models


In [0]:
from tensorflow.keras.losses import binary_crossentropy
from keras.optimizers import adam
from keras import backend as K
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)
from tensorflow.keras.layers import concatenate, Embedding, Dense, Dropout, Reshape, UpSampling2D, Conv2D, BatchNormalization, Activation, Input, Concatenate, LeakyReLU, Flatten, Lambda

In [0]:
dis_optimizer = adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
gen_optimizer = adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

In [0]:
def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss

In [0]:
ca_model = model.create_CA_model(len_embedding=1024)
ca_model.compile(loss="binary_crossentropy", optimizer="adam")

stage1_dis = model.create_disc_st1()
stage1_dis.compile(loss='binary_crossentropy', optimizer="adam")#dis_optimizer)

stage1_gen = model.create_gen_st1(len_embedding=1024)
stage1_gen.compile(loss="mse", optimizer="adam")#gen_optimizer)

#embedding_compressor_model = create_embedding_compressor()
#embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

adversarial_model = model.create_adversarial_model(gen=stage1_gen, disc=stage1_dis,len_embedding=1024)
adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                          optimizer="adam", metrics=None)#gen_optimizer, metrics=None)

# Train

Earlier, label/target values for a classifier were 0 or 1; 0 for fake images and 1 for real images. Because of this, GANs were prone to adversarial examples, which are inputs to a neural network that result in an incorrect output from the network. Label smoothing is an approach to provide smoothed labels to the discriminator network. This means we can have decimal values such as 0.9 (true), 0.8 (true), 0.1 (fake), or 0.2 (fake), instead of labeling every example as either 1 (true) or 0 (fake). We smooth the target values (label values) of the real images as well as of the fake images. Label smoothing can reduce the risk of adversarial examples in GANs. To apply label smoothing, assign the labels 0.9, 0.8, and 0.7, and 0.1, 0.2, and 0.3, to the images. To find out more about label smoothing, refer to the following paper.

[Improved techniques for training GAN](https://arxiv.org/pdf/1606.03498.pdf)

In [16]:
#emb_train=emb_train.to_numpy()
emb_train=embedding
emb_train.shape

(32364, 1024)

In [0]:
batch_size=64
epoch=50

real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1

for epoch in range(epochs):
  print("========================================")
  print("Epoch is:", epoch)
  print("Number of batches", int(img_train.shape[0] / batch_size))

  gen_losses = []
  dis_losses = []

  number_of_batches = int(img_train.shape[0] / batch_size)
  for index in range(number_of_batches):
    #print("Batch:{}".format(index+1))

     # Create a batch of noise vectors
    z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
    image_batch = img_train[index * batch_size:(index + 1) * batch_size]
    embedding_batch = emb_train[index * batch_size:(index + 1) * batch_size]

    # Normalize images
    image_batch = image_batch/255 #(image_batch - 127.5) / 127.5

    fake_images, _ = stage1_gen.predict([embedding_batch, z_noise], verbose=3)
    
    #embedding compression
    compressed_embedding = np.hstack((embedding_batch,embedding_batch))
    compressed_embedding = np.reshape(compressed_embedding, (batch_size, 4, 4, condition_dim))

    #calc losses
    dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],np.reshape(real_labels, (batch_size, 1)))
    dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],np.reshape(fake_labels, (batch_size, 1)))
    dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],np.reshape(fake_labels[1:], (batch_size-1, 1)))

    g_loss = adversarial_model.train_on_batch([embedding_batch, z_noise, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])

    d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))

    if index%100 ==0:
      print("Batch:{}".format(index+1))
      print("d_loss:{}".format(d_loss))
            
      print("g_loss:{}".format(g_loss))

    dis_losses.append(d_loss)
    gen_losses.append(g_loss)

Epoch is: 0
Number of batches 505
Batch:1
d_loss:6.790207892656326
g_loss:[1.9276200532913208, 0.33735978603363037, 0.7951301336288452]
Batch:101
d_loss:0.614503797609359
g_loss:[1.5474025011062622, 1.4953705072402954, 0.026015974581241608]
Batch:201
d_loss:0.6895494111813605
g_loss:[1.0857374668121338, 1.0500519275665283, 0.017842750996351242]
Batch:301
d_loss:0.5551434475637507
g_loss:[0.8775706887245178, 0.8452175855636597, 0.016176562756299973]
Batch:401
d_loss:0.5858903607877437
g_loss:[0.9433809518814087, 0.9161632657051086, 0.013608831912279129]
Batch:501
d_loss:0.577331855225566
g_loss:[0.8125073313713074, 0.7889384627342224, 0.011784447357058525]
Epoch is: 1
Number of batches 505
Batch:1
d_loss:0.6045834853011911
g_loss:[0.7950640320777893, 0.772757351398468, 0.011153333820402622]
Batch:101
d_loss:0.5817941618443001
g_loss:[0.7483903169631958, 0.7264274954795837, 0.010981416329741478]
Batch:201
d_loss:0.666309442882266
g_loss:[0.9171547293663025, 0.898461103439331, 0.009346826

In [0]:
def save_rgb_img(img, path):
    """
    Save a rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

In [34]:
z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
embedding_batch = emb_test[0:batch_size]
fake_images, _ = stage1_gen.predict_on_batch([embedding_batch, z_noise2])

# Save images
for i, img in enumerate(fake_images[:10]):
  save_rgb_img(img, "/content/drive/My Drive/fake_img/gen_{}_{}.png".format(epoch, i))

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [0]:
stage1_gen.save_weights("stage1_gen.h5")
stage1_dis.save_weights("stage1_dis.h5")

In [26]:
im.show(fake_images)

NameError: ignored

In [31]:
emb_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.021993,-0.333375,1.142767,-0.812596,-0.113876,0.661719,-0.586909,0.259515,-0.128141,0.663969,-0.515993,0.436046,-0.333097,0.102360,0.652380,-0.461109,0.027995,-0.125902,0.034412,1.048993,0.034781,0.435503,0.965708,0.199289,0.258940,-0.450650,0.066384,-0.065783,-0.649887,-0.114031,-0.131986,0.021596,0.520628,-0.056275,-0.183740,-1.301966,0.193906,0.106588,0.062884,0.063359,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.354675,-0.617412,0.919868,-0.882100,0.397901,0.397487,-0.379388,0.314216,0.100787,0.786078,-0.697638,0.121449,0.013268,0.239560,0.392050,-0.556672,0.719795,0.278738,-0.570492,0.837745,-0.194009,0.268559,0.380689,0.624278,-0.094663,-0.623941,0.133633,-0.217117,-0.672485,0.211056,1.322194,-0.119464,-0.388648,-0.748765,-0.199255,-0.382009,0.088932,-0.839256,0.784935,0.513740,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,-0.026921,-0.381943,1.107266,-0.429580,0.755200,-0.015292,-0.865784,0.681212,-0.146739,0.971243,-0.869253,0.158204,-1.006461,0.591428,0.781535,-0.288375,0.076625,0.338774,-0.468398,1.139519,0.426430,-0.685776,0.402645,-0.259806,-0.054182,-0.401925,-0.260160,-0.580926,-0.151273,-0.350646,1.168210,-0.291671,-0.126155,-0.602235,-0.296945,-0.798772,0.848600,-0.627088,0.538728,-0.287223,...,0.267439,0.249059,-0.030523,-0.085384,-0.040739,-0.081996,0.181996,-0.13307,0.006534,0.132713,-0.113397,-0.121518,0.201801,0.135771,0.258362,-0.250189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,-0.245155,-0.197599,1.299455,-0.923318,-0.386898,0.808841,-0.498440,0.298925,0.311916,0.352545,-0.435687,0.287018,0.030146,0.344678,0.067195,-0.603182,0.035969,0.046298,-0.068767,0.504222,-0.619470,0.480948,0.983274,0.454100,0.032968,-0.265197,0.233669,-0.356425,-0.802311,0.291545,-0.199171,0.468158,0.094786,-0.587081,-0.471789,-1.244216,0.200717,0.218923,0.434786,0.467018,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,-0.245155,-0.197599,1.299455,-0.923318,-0.386898,0.808841,-0.498440,0.298925,0.311916,0.352545,-0.435687,0.287018,0.030146,0.344678,0.067195,-0.603182,0.035969,0.046298,-0.068767,0.504222,-0.619470,0.480948,0.983274,0.454100,0.032968,-0.265197,0.233669,-0.356425,-0.802311,0.291545,-0.199171,0.468158,0.094786,-0.587081,-0.471789,-1.244216,0.200717,0.218923,0.434786,0.467018,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40430,-0.245155,-0.197599,1.299455,-0.923318,-0.386898,0.808841,-0.498440,0.298925,0.311916,0.352545,-0.435687,0.287018,0.030146,0.344678,0.067195,-0.603182,0.035969,0.046298,-0.068767,0.504222,-0.619470,0.480948,0.983274,0.454100,0.032968,-0.26